In [2]:
%pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 879.0/879.0 kB 50.9 MB/s eta 0:00:00


In [3]:
!gdown https://drive.google.com/uc?id=1Lm2dCB3BhYdng6Lgo00of5hPxrc0njy9 -O dataset.zip

import zipfile

with zipfile.ZipFile('dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('dataset')

print("File berhasil diunduh dan diekstrak.")

Downloading...
From (original): https://drive.google.com/uc?id=1Lm2dCB3BhYdng6Lgo00of5hPxrc0njy9
From (redirected): https://drive.google.com/uc?id=1Lm2dCB3BhYdng6Lgo00of5hPxrc0njy9&confirm=t&uuid=767f6f16-8f01-46c8-be81-2bc82ac5eca9
To: /content/dataset.zip
100% 203M/203M [00:02<00:00, 71.7MB/s]
File berhasil diunduh dan diekstrak.


In [4]:
%%writefile /content/custom_dataset.yaml
train: /content/dataset/yolo/train/images
val: /content/dataset/yolo/val/images

nc: 14
names:
- explorer
- search
- accounts
- manage
- source_control
- run_and_debug
- extensions
- activity_bar
- primary_sidebar
- editor
- status_bar
- close
- panel
- testing

# Pengaturan augmentasi
augment:
  # - type: hflip
  #   probability: 0.5
  # - type: vflip
  #   probability: 0.5
  # - type: rotate
  #   angle: 15
  #   probability: 0.5
  - type: scale
    factor: 0.5
    probability: 0.5
  - type: gauss_noise
    mean: 0
    std: 25
    probability: 0.5  # Menambahkan Gaussian noise
  - type: gaussian_blur
    kernel_size: 5
    probability: 0.5  # Menambahkan Gaussian blur
  - type: random_brightness
    factor: 0.2
    probability: 0.5  # Mengubah brightness
  - type: random_contrast
    factor: 0.2
    probability: 0.5  # Mengubah contrast
  - type: random_saturation
    factor: 0.2
    probability: 0.5  # Mengubah saturation
  - type: random_hue
    factor: 0.1
    probability: 0.5  # Mengubah hue

Writing /content/custom_dataset.yaml


In [2]:
from ultralytics import YOLO
import torch
import logging
import os

# Konfigurasi logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class HeadTrainer:
    def __init__(self, model_type='yolov8m.pt'):
        """
        Inisialisasi trainer dengan model YOLOv8 yang dipilih
        Args:
            model_type: Tipe model YOLOv8 ('yolov8n.pt', 'yolov8s.pt', 'yolov8m.pt', etc.)
        """
        self.model_type = model_type

    def prepare_model(self):
        """
        Memuat model YOLOv8 pretrained dan mempersiapkan untuk training
        """
        logger.info(f"Memuat model pretrained {self.model_type}...")

        # Load pretrained model
        self.model = YOLO(self.model_type)

        # Freeze backbone dan neck
        model_list = self.model.model.model
        for i in range(len(model_list) - 1):  # Semua layer kecuali head
            for param in model_list[i].parameters():
                param.requires_grad = False

        logger.info("Model siap untuk training (hanya head layer yang trainable)")

    def train(self, data_yaml, epochs=100, imgsz=640, batch=16):
        """
        Melatih head model dengan dataset yang diberikan
        Args:
            data_yaml: Path ke file konfigurasi dataset
            epochs: Jumlah epochs untuk training
            imgsz: Ukuran gambar input
            batch: Ukuran batch
        """
        if not os.path.exists(data_yaml):
            raise FileNotFoundError(f"File konfigurasi dataset tidak ditemukan: {data_yaml}")

        logger.info(f"Memulai training dengan {epochs} epochs...")

        # Training dengan ultralytics API
        results = self.model.train(
            data=data_yaml,
            epochs=epochs,
            imgsz=imgsz,
            batch=batch,
            project='head_training',
            name='exp',
            exist_ok=True
        )

        # Simpan head yang telah dilatih
        trained_head = self.model.model.model[-1]
        os.makedirs('trained_models', exist_ok=True)
        torch.save(trained_head, 'trained_models/trained_head.pt')
        logger.info("Training selesai. Head model disimpan di trained_models/trained_head.pt")

        return results

def main():
    # Inisialisasi trainer dengan model YOLOv8m
    trainer = HeadTrainer('yolov8m.pt')

    # Persiapkan model
    trainer.prepare_model()

    # Mulai training
    trainer.train(
        data_yaml='/content/custom_dataset.yaml',  # Ganti dengan path dataset Anda
        epochs=10,
        imgsz=640,
        batch=25
    )

if __name__ == "__main__":
    main()

Ultralytics 8.3.27 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/custom_dataset.yaml, epochs=10, time=None, patience=100, batch=25, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=head_training, name=exp, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, sh

train: Scanning /content/dataset/yolo/train/labels.cache... 480 images, 0 backgrounds, 0 corrupt: 100%|██████████| 480/480 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/dataset/yolo/val/labels.cache... 120 images, 0 backgrounds, 0 corrupt: 100%|██████████| 120/120 [00:00<?, ?it/s]


Plotting labels to head_training/exp/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000556, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005859375000000001), 83 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to head_training/exp
Starting training for 10 epochs...
Closing dataloader mosaic
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      10/10      11.5G      0.654     0.4169     0.8109         34        640: 100%|██████████| 20/20 [00:22<00:00,  1.13s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  1.62it/s]

                   all        120       1521      0.977       0.96      0.969      0.802



10 epochs completed in 0.066 hours.
Optimizer stripped from head_training/exp/weights/last.pt, 52.0MB
Optimizer stripped from head_training/exp/weights/best.pt, 52.0MB

Validating head_training/exp/weights/best.pt...
Ultralytics 8.3.27 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 218 layers, 25,847,866 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:03<00:00,  1.30s/it]


                   all        120       1521      0.976       0.96      0.969      0.802
              explorer        120        120      0.994          1      0.995       0.83
                search        120        120      0.996          1      0.995      0.819
              accounts        120        120      0.994          1      0.995      0.748
                manage        120        120      0.993          1      0.995      0.821
        source_control        120        120          1      0.997      0.995      0.763
         run_and_debug        120        120      0.997          1      0.995      0.784
            extensions        120        120      0.993          1      0.995      0.833
          activity_bar        120        120      0.993          1      0.995      0.877
       primary_sidebar        104        104      0.993          1      0.995      0.991
                editor        109        109       0.97          1      0.995      0.994
            status_ba

In [3]:
from ultralytics import YOLO

# Load the model
model = YOLO("/content/head_training/exp/weights/best.pt")

# Run prediction
results = model.predict(
    source="test.png",           # Path to your test image
    conf=0.1,                    # Confidence threshold (as a float, not a string)
    imgsz=640,                  # Image size (as an integer, not a string)
    save=True,                   # Save output images with predictions
    project="/content/output"    # Directory where the output will be saved
)


image 1/1 /content/test.png: 384x640 1 explorer, 1 search, 1 accounts, 1 manage, 2 source_controls, 2 run_and_debugs, 1 extensions, 1 activity_bar, 1 primary_sidebar, 1 editor, 1 panel, 2 testings, 47.8ms
Speed: 2.1ms preprocess, 47.8ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)
Results saved to /content/output/predict
